In [31]:
def output_matched_face(input_f,validated_f,output_f):
    import face_recognition
    from PIL import Image
    import os

    # Step 1: Load the validated image and encode the face
    validated_image = face_recognition.load_image_file(validated_f)
    validated_encodings = face_recognition.face_encodings(validated_image)

    if not validated_encodings:
        raise Exception("No face found in the validated image.")

    validated_encoding = validated_encodings[0]

    # Step 2: Load the second image (which might contain multiple faces)
    image_to_search = face_recognition.load_image_file(input_f)
    face_locations = face_recognition.face_locations(image_to_search)
    face_encodings = face_recognition.face_encodings(image_to_search, face_locations)
    image_height, image_width, _ = image_to_search.shape

    # Step 3: Output the face of the matched person
    # Add a margin around the person, or if only one person is detected, use the whole image

    def adjust_face_location(top, right, bottom, left, image_height, image_width, margin=0.2):
        
        width = right - left
        height = bottom - top

        new_top = max(0, top - int(height * margin))
        new_right = min(image_width, right + int(width * margin))
        new_bottom = min(image_height, bottom + int(height * margin))
        new_left = max(0, left - int(width * margin))

        return new_top, new_right, new_bottom, new_left

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        match = face_recognition.compare_faces([validated_encoding], face_encoding)
        if match[0]:
            
            # adjust the picture to just that person
            top, right, bottom, left = adjust_face_location(top, right, bottom, left, image_height, image_width)
            if len(face_locations) == 1:
                # but use whole picture if only one person is detected
                top, right, bottom, left = (0, image_width, image_height, 0)

            pil_image = Image.fromarray(image_to_search)
            face_image = pil_image.crop((left, top, right, bottom))
            
            os.makedirs(os.path.dirname(output_f), exist_ok=True)            
            face_image.save(output_f)
            break
    else:
        print(f"No matching face for {input_f}")

In [32]:
import pandas as pd
import os
import glob
import re
csv_filename = "ceo_data.csv"  # Change this to your actual file path
ceo_df = pd.read_csv(csv_filename)

for _, row in ceo_df[:1].iterrows():
    ceo = row["CEO"]

    validated_f = f"verified photos/{ceo}.jpg"
    if not os.path.exists(validated_f):
        print(f"Missing validated picture for {ceo}")
        continue

    pictures_files = glob.glob(f"pictures/{ceo}/*/*") 
    pictures_files = [f for f in pictures_files if os.path.isfile(f)]
    pictures_files = [f for f in pictures_files if not '.csv' in f]

    for f in pictures_files:
        year = f.split('\\')[-2]
        f_name = f.split('\\')[-1]
        print(year)
        # if this year obj is actually a year, proceed
        if re.match(r'^\d{4}$', year):
            output_f = f"matched_pictures/{ceo}/{year}/{f_name}"
            output_matched_face(f,validated_f,output_f)

Missing validated picture for Steve Jobs
